In [32]:
import pytz 
import pandas as pd 
from datetime import datetime, timedelta
from datetime import time as datetime_time
from helpers.constants import LOCAL_STORAGE_FOLDER
from sklearn.preprocessing import MinMaxScaler

normalize = False 
filename = "sep12.csv"
# Clean and normalize es data. Normalize means to put it between 0 and 1
utc_tz = pytz.timezone("UTC")
eastern_tz = pytz.timezone("US/Eastern")
df = pd.read_csv(LOCAL_STORAGE_FOLDER / f"es_data/{filename}", skiprows=2000000, nrows=1000000)
day_of_data = (
    pd.to_datetime(df.iloc[0]["ts_recv"], unit="ns")
    .tz_localize(utc_tz)
    .tz_convert(eastern_tz)
)

market_open = datetime_time(9, 30)
market_open_full_datetime_ns = (
    datetime.combine(day_of_data.date(), market_open)
    .astimezone(pytz.timezone("US/Eastern"))
    .timestamp()
) * 1e9
market_close = datetime_time(16, 0)
market_close_full_datetime_ns = (
    datetime.combine(day_of_data.date(), market_close)
    .astimezone(pytz.timezone("US/Eastern"))
    .timestamp()
) * 1e9

df1 = df[
    (market_open_full_datetime_ns <= df["ts_recv"])
    & (df["ts_recv"] <= market_close_full_datetime_ns)
]
df2 = df1[df1["action"] == "F"]
columns_to_keep = ["ts_recv", "price"]
df3 = df2[columns_to_keep]
df3["ts_recv"] = pd.to_datetime(df3["ts_recv"], unit="ns")
high = df3["price"].quantile(0.95)
low = df3["price"].quantile(0.05)
df4 = df3[(df3["price"] < high) & (df3["price"] > low)]
if normalize:
    scaler = MinMaxScaler()
    df4["price"] = scaler.fit_transform(df4[["price"]])
df4["ts_recv"] = df4["ts_recv"].apply(
    lambda time: time.tz_localize(utc_tz).tz_convert(eastern_tz)
)


In [42]:
df.ts_recv.min()

1694494800059424547

In [41]:
1694525400000000000 < 1694508357409140443

False